# Importing libraries and downloading packages

In [14]:
import nltk
import numpy as np

In [15]:
# downloading model to tokenize message
nltk.download('punkt')
# downloading stopwords
nltk.download('stopwords')
# downloading wordnet, which contains all lemmas of english language
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sshas\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sshas\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\sshas\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [16]:
from nltk.tokenize import word_tokenize

from nltk.corpus import stopwords
stop_words = stopwords.words('english')

from nltk.stem import WordNetLemmatizer

In [17]:
stop_words

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

# Function to clean text

In [18]:
def clean_corpus(corpus):
  # lowering every word in text
  corpus = [ doc.lower() for doc in corpus]
  cleaned_corpus = []
  
  stop_words = stopwords.words('english')
  wordnet_lemmatizer = WordNetLemmatizer()

  # iterating over every text
  for doc in corpus:
    # tokenizing text
    tokens = word_tokenize(doc)
    cleaned_sentence = [] 
    for token in tokens: 
      # removing stopwords, and punctuation
      if token not in stop_words and token.isalpha(): 
        # applying lemmatization
        cleaned_sentence.append(wordnet_lemmatizer.lemmatize(token)) 
    cleaned_corpus.append(' '.join(cleaned_sentence))
  return cleaned_corpus

# Loading and cleaning intents

In [19]:
!wget -O intents.json https://techlearn-cdn.s3.amazonaws.com/bs_swiggy_chatbot/intent.json

'wget' is not recognized as an internal or external command,
operable program or batch file.


In [20]:
import json
with open('D:/Project series/AI/intents.json', 'r') as file:
  intents = json.load(file)

In [21]:
corpus = []
tags = []

for intent in intents['intents']:
    # taking all patterns in intents to train a neural network
    for pattern in intent['patterns']:
        corpus.append(pattern)
        tags.append(intent['tag'])

In [22]:
cleaned_corpus = clean_corpus(corpus)
cleaned_corpus

['hi',
 'anyone',
 'hey',
 'hola',
 'hello',
 'good day',
 'bye',
 'see later',
 'goodbye',
 'nice chatting bye',
 'till next time',
 '',
 'thanks',
 'thank',
 'helpful',
 'awesome thanks',
 'thanks helping',
 'could help',
 'help provide',
 'helpful',
 'support offered',
 'please check order status',
 'able check order status',
 'help order status',
 'order status',
 'order',
 'food',
 'track order',
 'track food',
 'hi want cancel order',
 'want cancel order',
 'please cancel order',
 'cancel order',
 'want add delivery instruction',
 'please add delivery instruction',
 'include delivery instruction',
 'tell joke',
 'feeling bored',
 'joke please',
 'make laugh',
 'want laugh']

# Vectorizing intents

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(cleaned_corpus)

In [24]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder()
y = encoder.fit_transform(np.array(tags).reshape(-1,1))

# Training neural network

In [25]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout

model = Sequential([
                    Dense(128, input_shape=(X.shape[1],), activation='relu'),
                    Dropout(0.2),
                    Dense(64, activation='relu'),
                    Dropout(0.2),
                    Dense(y.shape[1], activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               5888      
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 520       
Total params: 14,664
Trainable params: 14,664
Non-trainable params: 0
_________________________________________________________________


In [26]:
history = model.fit(X.toarray(), y.toarray(), epochs=20, batch_size=1)

Epoch 1/20
41/41 [==============================] - 1s 932us/step - loss: 2.0794 - accuracy: 0.1560
Epoch 2/20
41/41 [==============================] - 0s 868us/step - loss: 1.9954 - accuracy: 0.3807
Epoch 3/20
41/41 [==============================] - 0s 900us/step - loss: 1.8791 - accuracy: 0.4855
Epoch 4/20
41/41 [==============================] - 0s 1ms/step - loss: 1.7974 - accuracy: 0.7783
Epoch 5/20
41/41 [==============================] - 0s 1ms/step - loss: 1.5951 - accuracy: 0.8161
Epoch 6/20
41/41 [==============================] - 0s 1ms/step - loss: 1.4102 - accuracy: 0.7952
Epoch 7/20
41/41 [==============================] - 0s 1ms/step - loss: 1.1715 - accuracy: 0.9501
Epoch 8/20
41/41 [==============================] - 0s 1ms/step - loss: 0.8933 - accuracy: 0.9519
Epoch 9/20
41/41 [==============================] - 0s 1ms/step - loss: 0.6436 - accuracy: 0.9804
Epoch 10/20
41/41 [==============================] - 0s 875us/step - loss: 0.4836 - accuracy: 0.9317
Epoch 11/20

# Classifying messages to intent

1. If the intent probability does not match with any intent, then send it to no answer.

2. Get Intent

3. Perform Action


In [27]:
# if prediction for every tag is low, then we want to classify that message as noanswer

INTENT_NOT_FOUND_THRESHOLD = 0.40

def predict_intent_tag(message):
  message = clean_corpus([message])
  X_test = vectorizer.transform(message)
  y = model.predict(X_test.toarray())
  
  # if probability of all intent is low, classify it as noanswer
  if y.max() < INTENT_NOT_FOUND_THRESHOLD:
    return 'noanswer'
  
  prediction = np.zeros_like(y[0])
  prediction[y.argmax()] = 1
  tag = encoder.inverse_transform([prediction])[0][0]
  return tag

print(predict_intent_tag('How you could help me?'))
print(predict_intent_tag('swiggy chat bot'))
print(predict_intent_tag('Where\'s my order'))


options
goodbye
order-status-request


In [28]:
import random
import time 

In [29]:
def get_intent(tag):
  # to return complete intent from intent tag
  for intent in intents['intents']:
    if intent['tag'] == tag:
      return intent

In [30]:
def perform_action(action_code, intent):
  # funition to perform an action which is required by intent
  
  if action_code == 'CHECK_ORDER_STATUS':
    print('\n Checking database \n')
    time.sleep(2)
    order_status = ['in kitchen', 'with delivery executive']
    delivery_time = []
    return {'intent-tag':intent['next-intent-tag'][0],
            'order_status': random.choice(order_status),
            'delivery_time': random.randint(10, 30)}
  
  elif action_code == 'ORDER_CANCEL_CONFIRMATION':
    ch = input('BOT: Do you want to continue (Y/n) ?')
    if ch == 'y' or ch == 'Y':
      choice = 0
    else:
      choice = 1
    return {'intent-tag':intent['next-intent-tag'][choice]}
  
  elif action_code == 'ADD_DELIVERY_INSTRUCTIONS':
    instructions = input('Your Instructions: ')
    return {'intent-tag':intent['next-intent-tag'][0]}

# Complete chat bot

In [ ]:
while True:
  # get message from user
  message = input('You: ')
  # predict intent tag using trained neural network
  tag = predict_intent_tag(message)
  # get complete intent from intent tag
  intent = get_intent(tag)
  # generate random response from intent
  response = random.choice(intent['responses'])
  print('Bot: ', response)

  # check if there's a need to perform some action
  if 'action' in intent.keys():
    action_code = intent['action']
    # perform action
    data = perform_action(action_code, intent)
    # get follow up intent after performing action
    followup_intent = get_intent(data['intent-tag'])
    # generate random response from follow up intent
    response = random.choice(followup_intent['responses'])

    # print randomly selected response
    if len(data.keys()) > 1:
      print('Bot: ', response.format(**data))
    else:
      print('Bot: ', response)

  # break loop if intent was goodbye
  if tag == 'goodbye':
    break

You: hello
Bot:  Hi there, how can I help?
You: I need my order
Bot:  I am checking your status of your order, Please wait.

 Checking database 

Bot:  Your order will be delivered in 19, currently it is with delivery executive. Is there anything else, we can help you with?
You: where is my order
Bot:  I am checking your status of your order, Please wait.

 Checking database 

Bot:  Your order will be delivered in 24, currently it is with delivery executive. Is there anything else, we can help you with?
You: I want to add some order
Bot:  Not sure I understand
